In [1]:
import pandas as pd
import numpy as np


In [2]:
csv_path = "flight_weather_merged.csv"
df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료")
print("행 수:", len(df))
print("컬럼 수:", df.shape[1])
df.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_17692\1893049337.py:2: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ CSV 로드 완료
행 수: 1262522
컬럼 수: 52


,AIRLINE_ENGLISH,AIRLINE_KOREAN,AIRPORT,AIR_FLN,ARRIVED_ENG,ARRIVED_KOR,BAGGAGE_CLAIM,BOARDING_ENG,BOARDING_KOR,CITY,...,3시간신적설(cm),중하층운량(10분위),현상번호(국내식),지면온도(°C),풍속_ms,풍향_deg,시정_m,전운량_okta,1층 운고(FT)_결측여부,weather_datetime
0,AIR SEOUL,에어서울,GMP,RS901,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
1,JINAIR,진에어,GMP,LJ301,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
2,ASIANA AIRLINE,아시아나항공,GMP,OZ8937,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
3,EASTAR JET,이스타항공,GMP,ZE201,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00
4,JINAIR,진에어,GMP,LJ303,JEJU,제주,NaN,GIMPO,김포,CJU,...,0.0,NaN,0.0,NaN,2.057776,240.0,10000.0,1.0,0.0,2023-04-30 06:00:00


In [3]:
# =====================================================
# 2️⃣ 타겟 변수 생성 (지연 여부)
# =====================================================
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)
print("\n📌 is_delay 분포")
print(df["is_delay"].value_counts())


📌 is_delay 분포
0    1250909
1      11613
Name: is_delay, dtype: int64


In [4]:
# =====================================================
# 3️⃣ 날짜 / 시간 파생 (기존 방식 유지)
# =====================================================
df["flight_datetime"] = pd.to_datetime(df["flight_datetime"])

df["flight_date"] = df["flight_datetime"].dt.date
df["weekday"] = df["flight_datetime"].dt.weekday
df["month"] = df["flight_datetime"].dt.month
df["hour"] = df["flight_datetime"].dt.hour


In [5]:
# =====================================================
# 4️⃣ 모델 입력에서 제외할 컬럼 정의
# =====================================================
exclude_cols = [
    # 타겟
    "is_delay",

    # 원본 날짜/시간
    "FLIGHT_DATE",
    "flight_date",
    "STD",
    "ETD",
    "flight_datetime",
    "weather_datetime",
    "일시",

    # 텍스트 설명 (누수 위험)
    "RMK_KOR",

    # 중복 / 의미 없는 식별자
    "지점명"
]

exclude_cols = [c for c in exclude_cols if c in df.columns]

In [6]:
# =====================================================
# 5️⃣ 범주형 / 수치형 컬럼 (🔥 개선된 명시적 선택)
# =====================================================

# ✅ 범주형 (flight 핵심 + weather 핵심)
cat_cols = [
    # flight
    "AIRLINE_KOREAN",
    "BOARDING_KOR",
    "ARRIVED_KOR",
    "LINE",
    "IO",

    # weather (범주형)
    "일기현상",
    "1층 운형"
]

cat_cols = [c for c in cat_cols if c in df.columns]

# ✅ 수치형 (시간 파생 + 날씨 수치형)
numeric_candidates = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

# 숫자처럼 생긴 ID / 코드 제거
exclude_numeric_like_id = [
    "지점",
    "현상번호(국내식)"
]

num_cols = [
    c for c in numeric_candidates
    if c not in exclude_numeric_like_id
    and c not in exclude_cols
]

print("\n📊 최종 수치형 컬럼 수:", len(num_cols))
print(num_cols)

print("\n🏷 최종 범주형 컬럼 수:", len(cat_cols))
print(cat_cols)


📊 최종 수치형 컬럼 수: 24
['STD_clean', '1층 운량(1/8)', '1층 운고(FT)', '기온(°C)', '이슬점온도(°C)', '해면기압(hPa)', '현지기압(hPa)', '강수량(mm)', '습도(%)', '증기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '중하층운량(10분위)', '지면온도(°C)', '풍속_ms', '풍향_deg', '시정_m', '전운량_okta', '1층 운고(FT)_결측여부', 'weekday', 'month', 'hour']

🏷 최종 범주형 컬럼 수: 7
['AIRLINE_KOREAN', 'BOARDING_KOR', 'ARRIVED_KOR', 'LINE', 'IO', '일기현상', '1층 운형']


In [7]:
# =====================================================
# 6️⃣ 결측치 처리
# =====================================================

# 수치형 → 중앙값
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# 범주형 → UNKNOWN
for col in cat_cols:
    df[col] = df[col].fillna("UNKNOWN")

In [8]:
# =====================================================
# 7️⃣ Train / Test 분리 (날짜 기준)
# =====================================================
df = df.sort_values("flight_date")

split_date = df["flight_date"].quantile(0.8)

train_df = df[df["flight_date"] <= split_date]
test_df  = df[df["flight_date"] > split_date]

print("\n📦 데이터 분리")
print("Train:", len(train_df), "Test:", len(test_df))

X_cols = num_cols + cat_cols

X_train = train_df[X_cols]
y_train = train_df["is_delay"]

X_test  = test_df[X_cols]
y_test  = test_df["is_delay"]


📦 데이터 분리
Train: 1010666 Test: 251856


In [13]:
# =====================================================
# 🔍 A️⃣ 수치형 변수 ↔ is_delay 상관관계
# =====================================================
from scipy.stats import pointbiserialr

num_corr_results = []

for col in num_cols:
    try:
        corr, pval = pointbiserialr(train_df[col], train_df["is_delay"])
        num_corr_results.append({
            "feature": col,
            "corr": corr,
            "abs_corr": abs(corr),
            "p_value": pval
        })
    except Exception as e:
        print(f"⚠️ {col} 계산 실패:", e)

num_corr_df = (
    pd.DataFrame(num_corr_results)
    .sort_values("abs_corr", ascending=False)
)

print("\n📊 수치형 변수 상관관계 (상위 20개)")
display(num_corr_df.head(20))



📊 수치형 변수 상관관계 (상위 20개)


,feature,corr,abs_corr,p_value
12,적설(cm),0.117307,0.117307,0.000000e+00
16,풍속_ms,0.090232,0.090232,0.000000e+00
3,기온(°C),-0.061659,0.061659,0.000000e+00
19,전운량_okta,0.058766,0.058766,0.000000e+00
7,강수량(mm),0.045205,0.045205,0.000000e+00
0,STD_clean,0.043898,0.043898,0.000000e+00
23,hour,0.043756,0.043756,0.000000e+00
2,1층 운고(FT),-0.037164,0.037164,1.032203e-305
4,이슬점온도(°C),-0.035587,0.035587,1.728303e-280
8,습도(%),0.032422,0.032422,3.725727e-233


In [14]:
# =====================================================
# 🔍 B️⃣ 범주형 변수 ↔ is_delay (Chi-square)
# =====================================================
from scipy.stats import chi2_contingency

cat_corr_results = []

for col in cat_cols:
    contingency = pd.crosstab(train_df[col], train_df["is_delay"])

    # 모든 값이 한쪽 클래스만 가지면 스킵
    if contingency.shape[1] < 2:
        continue

    chi2, p, _, _ = chi2_contingency(contingency)

    cat_corr_results.append({
        "feature": col,
        "chi2": chi2,
        "p_value": p
    })

cat_corr_df = (
    pd.DataFrame(cat_corr_results)
    .sort_values("chi2", ascending=False)
)

print("\n🏷 범주형 변수 Chi-square 결과")
display(cat_corr_df)



🏷 범주형 변수 Chi-square 결과


,feature,chi2,p_value
5,일기현상,42038.316097,0.000000e+00
6,1층 운형,11284.296719,0.000000e+00
2,ARRIVED_KOR,3495.183403,0.000000e+00
1,BOARDING_KOR,2921.059564,0.000000e+00
0,AIRLINE_KOREAN,2316.548932,0.000000e+00
3,LINE,651.865682,8.777342e-144
4,IO,238.725957,7.456014e-54


In [9]:
# =====================================================
# 8️⃣ 전처리 + 모델 정의
# =====================================================
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

class_weight = {0: 1, 1: 70}

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)
    ]
)

model = Pipeline([
    ("prep", preprocessor),
    ("clf", LogisticRegression(
        max_iter=5000,
        class_weight=class_weight,
        n_jobs=-1
    ))
])

In [10]:
# =====================================================
# 9️⃣ 모델 학습
# =====================================================
model.fit(X_train, y_train)
print("\n✅ 모델 학습 완료")


✅ 모델 학습 완료


In [11]:
# =====================================================
# 🔟 평가
# =====================================================
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    average_precision_score
)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("\n📊 기본 성능")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC :", average_precision_score(y_test, y_prob))


📊 기본 성능
              precision    recall  f1-score   support

           0       1.00      0.90      0.95    250885
           1       0.02      0.43      0.03       971

    accuracy                           0.90    251856
   macro avg       0.51      0.67      0.49    251856
weighted avg       0.99      0.90      0.95    251856

ROC-AUC: 0.7522706262467324
PR-AUC : 0.022187775040216948


In [12]:
# =====================================================
# 1️⃣1️⃣ Threshold 실험
# =====================================================
for t in [0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f"\n===== Threshold = {t} =====")
    y_pred_t = (y_prob >= t).astype(int)
    print(classification_report(y_test, y_pred_t))


===== Threshold = 0.3 =====
              precision    recall  f1-score   support

           0       1.00      0.72      0.84    250885
           1       0.01      0.66      0.02       971

    accuracy                           0.72    251856
   macro avg       0.50      0.69      0.43    251856
weighted avg       0.99      0.72      0.83    251856


===== Threshold = 0.4 =====
              precision    recall  f1-score   support

           0       1.00      0.83      0.91    250885
           1       0.01      0.54      0.02       971

    accuracy                           0.83    251856
   macro avg       0.51      0.68      0.47    251856
weighted avg       0.99      0.83      0.90    251856


===== Threshold = 0.5 =====
              precision    recall  f1-score   support

           0       1.00      0.90      0.95    250885
           1       0.02      0.43      0.03       971

    accuracy                           0.90    251856
   macro avg       0.51      0.67      0.